In [1]:
#creating soup for games on a given day
import re #regular expressions package
import requests
import httplib2 #http library
from bs4 import BeautifulSoup #preprocesses soup for quicker speeds
from bs4 import SoupStrainer
import time #used for delaying system
from datetime import datetime #used to format date/time
import mysql.connector

#define function for scraping
http = httplib2.Http()

In [2]:
# log of scraped days
# 10/2022

In [3]:
##generate links for a given day
#month = '1'
#day = '14'
#year = '2023'
#status, response = http.request(f'https://www.basketball-reference.com/boxscores/?month={month}&day={day}&year={year}')

#soup3 = BeautifulSoup(response, "html.parser", parse_only=SoupStrainer('a'))

##create object containing links
#links = []
#for link in soup3.find_all('a', href=re.compile(f'^/boxscores/{year}')):
    #path = link['href']
    #links.append(f'https://www.basketball-reference.com{path}')
#links = links[1::2]
#print(links)

In [4]:
#url = 'https://www.basketball-reference.com/boxscores/?month=01&day=28&year=2023'
#page = requests.get(url, allow_redirects=False)

#print(page.text)

In [5]:
#generate links for a range of days
links = []
month = '10'
year = '2022'
for x in range(18,31):
    time.sleep(3.5)
    status, response = http.request(f'https://www.basketball-reference.com/boxscores/?month={month}&day={x}&year={year}')

    soup4 = BeautifulSoup(response, "html.parser", parse_only=SoupStrainer('a'))
    for link in soup4.find_all('a', href=re.compile(f'^/boxscores/{year}')):
        path = link['href']
        links.append(f'https://www.basketball-reference.com{path}')
links = links[1::2]

In [6]:
print(len(links))
for x in links:
    print(x)

95
https://www.basketball-reference.com/boxscores/202210180BOS.html
https://www.basketball-reference.com/boxscores/202210180GSW.html
https://www.basketball-reference.com/boxscores/202210190ATL.html
https://www.basketball-reference.com/boxscores/202210190BRK.html
https://www.basketball-reference.com/boxscores/202210190DET.html
https://www.basketball-reference.com/boxscores/202210190IND.html
https://www.basketball-reference.com/boxscores/202210190MEM.html
https://www.basketball-reference.com/boxscores/202210190MIA.html
https://www.basketball-reference.com/boxscores/202210190MIN.html
https://www.basketball-reference.com/boxscores/202210190PHO.html
https://www.basketball-reference.com/boxscores/202210190SAC.html
https://www.basketball-reference.com/boxscores/202210190SAS.html
https://www.basketball-reference.com/boxscores/202210190TOR.html
https://www.basketball-reference.com/boxscores/202210190UTA.html
https://www.basketball-reference.com/boxscores/202210200LAL.html
https://www.basketball

In [7]:
cnx = mysql.connector.connect(user='nba_user', password='letmein',
                              host='127.0.0.1',
                              database='nba_data')
cursor = cnx.cursor()

In [8]:
#create function used to push data to mysql
sql = ("INSERT IGNORE INTO nba (game_time, game_date, arena, home_team, home_wins, home_losses, away_team, away_wins, away_losses, attendance)" 
       "VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)")

In [9]:
gamesdata=[]
for x in links:
    #creates buffer every 3.5 seconds
    time.sleep(3.5)
    #extracts text html from site
    URL = x
    page = requests.get(URL, allow_redirects=False)
    #creates object in BS containing html from site
    soup = BeautifulSoup(page.content, "html.parser")

    #create list to begin adding information to
    gamedata = []

    #create list with time, date, and venue info
    tdv = []
    for div in soup.find_all("div", {"class": "scorebox_meta"}):
        for childdiv in div.find_all('div'):
            tdv.append(childdiv.string)

    #convert date/time to usable format
    date_str = tdv[0]
    
    # %p is to parse Am/Pm
    tdv[0] = datetime.strptime(date_str, "%I:%M %p, %B %d, %Y").strftime("%H:%M:%S %Y-%m-%d")

    #split time/date and isolate arena
    timedate = tdv[0].split(" ",1)
    venue = tdv[1].split(",",1)

    #append timedate and venue to gamedata
    gamedata.append(timedate[0])
    gamedata.append(timedate[1])
    gamedata.append(venue[0])

    #create list containing teams playing
    teams = []
    for strong_tag in soup.find_all('strong', limit=3):
        stuff = (strong_tag.text)
        teams.append(stuff)
    teams[1] = teams[1].replace('\n','')
    teams[2] = teams[2].replace('\n','')

    #create list containing records
    records = []
    for div in soup.find_all("div", {"class": "scores"}):
        records.append(div.next_sibling.get_text(' '))

    #split records into home/away
    vrec = records[0].split("-",1)
    hrec = records[1].split("-",1)

    #append teams and records to gamedata
    gamedata.append(teams[2])
    gamedata.append(hrec[0])
    gamedata.append(hrec[1])
    gamedata.append(teams[1])
    gamedata.append(vrec[0])
    gamedata.append(vrec[1])

    #create object containing attendance
    attendance = []
    for strong_tag in soup.find_all('strong', limit=8):
        stuff = (strong_tag.next_sibling)
        attendance.append(stuff)

    #isolate attendance
    attendance = attendance[7].replace(',','')

    #append attendance to gamedata
    gamedata.append(attendance)
    gamesdata.append(gamedata)
    
    #create objects for pushing into database
    game_time = gamedata[0]
    game_date = gamedata[1]
    arena = gamedata[2]
    home_team = gamedata[3]
    home_wins = gamedata[4]
    home_losses = gamedata[5]
    away_team = gamedata[6]
    away_wins = gamedata[7]
    away_losses = gamedata[8]
    attendance = gamedata[9]
    
    #create tuple to insert into mysql
    val = (game_time, game_date, arena, home_team, home_wins, home_losses, away_team, away_wins, away_losses, attendance)
    
    #commit data to mysql
    cursor.execute(sql, val)

    cnx.commit()

In [ ]:
print(len(gamesdata))
for x in gamesdata:
    print(x)

95
['19:30:00', '2022-10-18', 'TD Garden', 'Boston Celtics', '1', '0', 'Philadelphia 76ers', '0', '1', '19156']
['22:00:00', '2022-10-18', 'Chase Center', 'Golden State Warriors', '1', '0', 'Los Angeles Lakers', '0', '1', '18064']
['19:30:00', '2022-10-19', 'State Farm Arena', 'Atlanta Hawks', '1', '0', 'Houston Rockets', '0', '1', '17878']
['19:30:00', '2022-10-19', 'Barclays Center', 'Brooklyn Nets', '0', '1', 'New Orleans Pelicans', '1', '0', '18003']
['19:00:00', '2022-10-19', 'Little Caesars Arena', 'Detroit Pistons', '1', '0', 'Orlando Magic', '0', '1', '20190']
['19:00:00', '2022-10-19', 'Gainbridge Fieldhouse', 'Indiana Pacers', '0', '1', 'Washington Wizards', '1', '0', '15027']
['19:30:00', '2022-10-19', 'FedEx Forum', 'Memphis Grizzlies', '1', '0', 'New York Knicks', '0', '1', '18202']
['19:30:00', '2022-10-19', 'FTX Arena', 'Miami Heat', '0', '1', 'Chicago Bulls', '1', '0', '19600']
['20:00:00', '2022-10-19', 'Target Center', 'Minnesota Timberwolves', '1', '0', 'Oklahoma Cit

In [84]:
#### cursor.close()
cnx.close()